# Gridded VIIRS Dataset

This notebook is for transforming the VIIRS (Visable Infrared Imaging Radiometer Suite) data for any given range of times when data is available, and
applying a grid mapping to datapoints within the region of interest. We are interested in northern California as that particular region expeirences frequent and intense wilfires. This notebook will take a CSV VIIRS dataframe and transform it into a 2D Xarray dataset with a grid mapping. 

In [1]:
#Import Libraries and Define VIIRS CSV

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapely.geometry
import geopandas as gpd
import xarray as xr
import argparse
import glob

ds = pd.read_csv("~/work/VIIRS/viirs-snpp_2018_United_States.csv")
ds.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,41.467583,-83.365883,321.32,0.43,0.38,2018-01-01,716,N,VIIRS,n,1,261.53,2.89,N,2
1,40.395264,-79.857147,329.87,0.39,0.36,2018-01-01,716,N,VIIRS,n,1,261.97,3.85,N,2
2,41.660519,-87.424576,297.55,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,257.88,1.38,N,3
3,41.638611,-87.144012,321.14,0.41,0.45,2018-01-01,716,N,VIIRS,n,1,260.90,4.35,N,3
4,41.681416,-87.421059,298.91,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,261.50,0.89,N,3


In [2]:
import geopandas
from shapely.geometry import Polygon

gdf = geopandas.GeoDataFrame(ds, geometry =geopandas.points_from_xy(ds.longitude, ds.latitude),
            crs="+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

In [3]:
gdf

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,geometry
0,41.467583,-83.365883,321.32,0.43,0.38,2018-01-01,716,N,VIIRS,n,1,261.53,2.89,N,2,POINT (-83.366 41.468)
1,40.395264,-79.857147,329.87,0.39,0.36,2018-01-01,716,N,VIIRS,n,1,261.97,3.85,N,2,POINT (-79.857 40.395)
2,41.660519,-87.424576,297.55,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,257.88,1.38,N,3,POINT (-87.425 41.661)
3,41.638611,-87.144012,321.14,0.41,0.45,2018-01-01,716,N,VIIRS,n,1,260.90,4.35,N,3,POINT (-87.144 41.639)
4,41.681416,-87.421059,298.91,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,261.50,0.89,N,3,POINT (-87.421 41.681)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552118,36.496708,-119.602585,345.06,0.60,0.53,2018-12-31,2136,N,VIIRS,n,1,287.59,18.91,D,0,POINT (-119.603 36.497)
552119,42.847439,-124.086708,334.68,0.41,0.45,2018-12-31,2137,N,VIIRS,n,1,278.48,6.84,D,0,POINT (-124.087 42.847)
552120,42.847839,-124.082092,336.93,0.41,0.45,2018-12-31,2137,N,VIIRS,n,1,276.68,6.84,D,0,POINT (-124.082 42.848)
552121,42.846310,-124.085411,337.47,0.41,0.45,2018-12-31,2137,N,VIIRS,n,1,279.79,5.62,D,0,POINT (-124.085 42.846)


In [4]:
gdf.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,geometry
0,41.467583,-83.365883,321.32,0.43,0.38,2018-01-01,716,N,VIIRS,n,1,261.53,2.89,N,2,POINT (-83.366 41.468)
1,40.395264,-79.857147,329.87,0.39,0.36,2018-01-01,716,N,VIIRS,n,1,261.97,3.85,N,2,POINT (-79.857 40.395)
2,41.660519,-87.424576,297.55,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,257.88,1.38,N,3,POINT (-87.425 41.661)
3,41.638611,-87.144012,321.14,0.41,0.45,2018-01-01,716,N,VIIRS,n,1,260.90,4.35,N,3,POINT (-87.144 41.639)
4,41.681416,-87.421059,298.91,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,261.50,0.89,N,3,POINT (-87.421 41.681)


In [5]:
gdf.total_bounds

array([-174.155441,   19.011648,  -67.073578,   70.405228])

In [6]:
gdf.to_file('~/work/VIIRS/VIIRS_Input/VIIRS_2018.shp') 

In [8]:

testing = gdf[:201] 

In [9]:
testing

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,geometry
0,41.467583,-83.365883,321.32,0.43,0.38,2018-01-01,716,N,VIIRS,n,1,261.53,2.89,N,2,POINT (-83.366 41.468)
1,40.395264,-79.857147,329.87,0.39,0.36,2018-01-01,716,N,VIIRS,n,1,261.97,3.85,N,2,POINT (-79.857 40.395)
2,41.660519,-87.424576,297.55,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,257.88,1.38,N,3,POINT (-87.425 41.661)
3,41.638611,-87.144012,321.14,0.41,0.45,2018-01-01,716,N,VIIRS,n,1,260.90,4.35,N,3,POINT (-87.144 41.639)
4,41.681416,-87.421059,298.91,0.42,0.45,2018-01-01,716,N,VIIRS,n,1,261.50,0.89,N,3,POINT (-87.421 41.681)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,19.380262,-155.069565,343.58,0.40,0.37,2018-01-01,2338,N,VIIRS,n,1,306.04,7.48,D,0,POINT (-155.070 19.380)
197,19.377449,-155.065216,351.06,0.40,0.37,2018-01-01,2338,N,VIIRS,n,1,313.66,15.90,D,0,POINT (-155.065 19.377)
198,19.377993,-155.061417,338.63,0.40,0.37,2018-01-01,2338,N,VIIRS,n,1,300.19,8.96,D,0,POINT (-155.061 19.378)
199,19.384157,-155.066315,342.84,0.40,0.37,2018-01-01,2338,N,VIIRS,n,1,311.33,7.48,D,0,POINT (-155.066 19.384)


In [10]:
#testing.to_file('VIIRS_Test.shp')

In [11]:
testing.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   latitude    201 non-null    float64 
 1   longitude   201 non-null    float64 
 2   bright_ti4  201 non-null    float64 
 3   scan        201 non-null    float64 
 4   track       201 non-null    float64 
 5   acq_date    201 non-null    object  
 6   acq_time    201 non-null    int64   
 7   satellite   201 non-null    object  
 8   instrument  201 non-null    object  
 9   confidence  201 non-null    object  
 10  version     201 non-null    int64   
 11  bright_ti5  201 non-null    float64 
 12  frp         201 non-null    float64 
 13  daynight    201 non-null    object  
 14  type        201 non-null    int64   
 15  geometry    201 non-null    geometry
dtypes: float64(7), geometry(1), int64(3), object(5)
memory usage: 25.2+ KB


In [12]:
dtest = pd.DataFrame(testing)

In [18]:
dtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   latitude    201 non-null    float64 
 1   longitude   201 non-null    float64 
 2   bright_ti4  201 non-null    float64 
 3   scan        201 non-null    float64 
 4   track       201 non-null    float64 
 5   acq_date    201 non-null    object  
 6   acq_time    201 non-null    int64   
 7   satellite   201 non-null    object  
 8   instrument  201 non-null    object  
 9   confidence  201 non-null    object  
 10  version     201 non-null    int64   
 11  bright_ti5  201 non-null    float64 
 12  frp         201 non-null    float64 
 13  daynight    201 non-null    object  
 14  type        201 non-null    int64   
 15  geometry    201 non-null    geometry
dtypes: float64(7), geometry(1), int64(3), object(5)
memory usage: 25.2+ KB


In [25]:
dtest2 = dtest.drop(columns = ['geometry'])

In [26]:
dtest2.to_csv("C:/Users/iontrap/Desktop/WES/USRA/grid_VIIRS/grid_test3.csv")